## REACT AGENT: End-to-end example


In [ ]:
# notebook should be run in the /django-app directory, with venv compiled from poetry

In [ ]:
from redbox.models.chain import AISettings
from redbox.chains.components import get_chat_llm
from redbox.models.settings import ChatLLMBackend
from redbox.models.settings import Settings
from redbox.models.file import ChunkResolution
from langgraph.prebuilt import create_react_agent, ToolNode
from langgraph.prebuilt.chat_agent_executor import AgentState
from uuid import uuid4
from langgraph.checkpoint.memory import MemorySaver
from redbox.graph.nodes.tools import build_search_wikipedia_tool

In [ ]:
# Redbox state and query - modified from chain.py
# changing classes from Basemodel to Typeddict or AgentState

from typing import TypedDict
from pydantic import Field
from uuid import UUID
from typing import Literal
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages


class ChainChatMessage(TypedDict):
    role: Literal["user", "ai", "system"]
    text: str


class MyRedboxQuery(TypedDict):
    question: str = Field(description="The last user chat message")
    s3_keys: list[str] = Field(description="List of files to process", default_factory=list)
    user_uuid: UUID = Field(description="User the chain in executing for")
    chat_history: list[ChainChatMessage] = Field(description="All previous messages in chat (excluding question)")
    ai_settings: AISettings = Field(description="User request AI settings", default_factory=AISettings)
    permitted_s3_keys: list[str] = Field(description="List of permitted files for response", default_factory=list)


class MyRedboxState(AgentState):
    # class MyRedboxState(BaseModel):
    request: MyRedboxQuery
    config: dict
    # documents: Annotated[DocumentState, document_reducer] = DocumentState()
    # route_name: str | None = None
    # metadata: Annotated[RequestMetadata | None, metadata_reducer] = None
    # citations: list[Citation] | None = None
    # steps_left: Annotated[int | None, RemainingStepsManager] = None
    messages: Annotated[list[AnyMessage], add_messages] = Field(default_factory=list)
    # is_last_step: str

    @property
    def last_message(self) -> AnyMessage:
        if not self.messages:
            raise ValueError("No messages in the state")
        return self.messages[-1]

In [ ]:
# Tools - adjusting code in tools.py script
# changes: accessing states like this: state[] since we are using TypedDict instead of  Basemodel

from typing import Annotated, Union

import numpy as np
import requests
from elasticsearch import Elasticsearch
from opensearchpy import OpenSearch
from langchain_core.documents import Document
from langchain_core.embeddings.embeddings import Embeddings
from langchain_core.tools import Tool, tool
from langgraph.prebuilt import InjectedState
from sklearn.metrics.pairwise import cosine_similarity

from redbox.api.format import format_documents
from redbox.chains.components import get_embeddings
from redbox.models.file import ChunkCreatorType, ChunkMetadata
from redbox.models.settings import get_settings
from redbox.retriever.queries import (
    add_document_filter_scores_to_query,
    build_document_query,
)
from redbox.retriever.retrievers import query_to_documents
from redbox.transform import merge_documents, sort_documents, bedrock_tokeniser


def build_search_documents_tool(
    es_client: Union[Elasticsearch, OpenSearch],
    index_name: str,
    embedding_model: Embeddings,
    embedding_field_name: str,
    chunk_resolution: ChunkResolution | None,
) -> Tool:
    """Constructs a tool that searches the index and sets state.documents."""

    @tool(response_format="content_and_artifact")
    def _search_documents(query: str, state: Annotated[MyRedboxState, InjectedState]) -> tuple[str, list[Document]]:
        """
        Search for documents uploaded by the user based on a query string.

        This function performs a search over the user's uploaded documents
        and returns snippets from the documents ordered by relevance and
        grouped by document.

        Args:
            query (str): The search query string used to match documents.
                This could be a keyword, phrase, question, or text from
                the documents.

        Returns:
            dict[str, Any]: A collection of document objects that match the query.
        """
        query_vector = state["config"]["embedding_model"].embed_query(query)
        selected_files = state["request"]["s3_keys"]
        permitted_files = state["request"]["permitted_s3_keys"]
        ai_settings = state["request"]["ai_settings"]

        # Initial pass
        initial_query = build_document_query(
            query=query,
            query_vector=query_vector,
            selected_files=selected_files,
            permitted_files=permitted_files,
            embedding_field_name=state["config"]["embedding_field_name"],
            chunk_resolution=state["config"]["chunk_resolution"],
            ai_settings=ai_settings,
        )
        initial_documents = query_to_documents(
            es_client=state["config"]["es_client"], index_name=state["config"]["index_name"], query=initial_query
        )

        # Handle nothing found (as when no files are permitted)
        if not initial_documents:
            return "", []

            # Adjacent documents
        with_adjacent_query = add_document_filter_scores_to_query(
            elasticsearch_query=initial_query,
            ai_settings=ai_settings,
            centres=initial_documents,
        )
        adjacent_boosted = query_to_documents(
            es_client=state["config"]["es_client"], index_name=state["config"]["index_name"], query=with_adjacent_query
        )

        # Merge and sort
        merged_documents = merge_documents(initial=initial_documents, adjacent=adjacent_boosted)
        sorted_documents = sort_documents(documents=merged_documents)

        # Return as state update
        return format_documents(sorted_documents), sorted_documents

    return _search_documents


def build_govuk_search_tool(filter=True) -> Tool:
    """Constructs a tool that searches gov.uk and sets state["documents"]."""

    tokeniser = bedrock_tokeniser

    def recalculate_similarity(response, query, num_results):
        embedding_model = get_embeddings(get_settings())
        em_query = embedding_model.embed_query(query)
        for r in response.get("results"):
            description = r.get("description")
            em_des = embedding_model.embed_query(description)
            r["similarity"] = cosine_similarity(np.array(em_query).reshape(1, -1), np.array(em_des).reshape(1, -1))[0][
                0
            ]
        response["results"] = sorted(response.get("results"), key=lambda x: x["similarity"], reverse=True)[:num_results]
        return response

    @tool(response_format="content_and_artifact")
    def _search_govuk(query: str, state: Annotated[MyRedboxState, InjectedState]) -> tuple[str, list[Document]]:
        """
        Search for documents on gov.uk based on a query string.
        This endpoint is used to search for documents on gov.uk. There are many types of documents on gov.uk.
        Types include:
        - guidance
        - policy
        - legislation
        - news
        - travel advice
        - departmental reports
        - statistics
        - consultations
        - appeals
        """

        url_base = "https://www.gov.uk"
        required_fields = [
            "format",
            "title",
            "description",
            "indexable_content",
            "link",
        ]
        ai_settings = state["request"]["ai_settings"]
        response = requests.get(
            f"{url_base}/api/search.json",
            params={
                "q": query,
                "count": (
                    ai_settings.tool_govuk_retrieved_results if filter else ai_settings.tool_govuk_returned_results
                ),
                "fields": required_fields,
            },
            headers={"Accept": "application/json"},
        )
        response.raise_for_status()
        response = response.json()

        if filter:
            response = recalculate_similarity(response, query, ai_settings.tool_govuk_returned_results)

        mapped_documents = []
        for i, doc in enumerate(response["results"]):
            if any(field not in doc for field in required_fields):
                continue

            mapped_documents.append(
                Document(
                    page_content=doc["indexable_content"],
                    metadata=ChunkMetadata(
                        index=i,
                        uri=f"{url_base}{doc['link']}",
                        token_count=tokeniser(doc["indexable_content"]),
                        creator_type=ChunkCreatorType.gov_uk,
                    ).model_dump(),
                )
            )

        return format_documents(mapped_documents), mapped_documents

    return _search_govuk

In [ ]:
def get_query(user_uuid, prompts, documents, ai_setting):
    redbox_query = MyRedboxQuery(
        question=f"{prompts[-1]}",
        s3_keys=documents,
        user_uuid=user_uuid,
        chat_history=prompts[:-1],
        ai_settings=ai_setting,
        permitted_s3_keys=documents,
    )
    return redbox_query

## ADD YOU DOCUMENTS AND QUESTION BELOW

In [ ]:
ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
documents = ["natasha.boyse@digital.trade.gov.uk/The_impact_of_AI_on_UK_jobs_and_training_report2.pdf"]
user_query = "What is the impact of AI on UK jobs?"
# user_query = 'Who is Putin?'
# user_query = 'How to renew my passport?'
redbox_query = get_query(uuid4(), prompts=[user_query], documents=documents, ai_setting=ai_setting)

In [ ]:
redbox_query

In [ ]:
from langchain_community.embeddings import BedrockEmbeddings

embedding_model = BedrockEmbeddings(region_name="eu-west-2", model_id="amazon.titan-embed-text-v2:0")

In [ ]:
# Tools

_env = Settings()

search_documents = build_search_documents_tool(
    es_client=_env.elasticsearch_client(),
    index_name=_env.elastic_chunk_alias,
    embedding_model=embedding_model,
    embedding_field_name=_env.embedding_document_field_name,  # not used
    chunk_resolution=ChunkResolution.normal,
)
search_wikipedia = build_search_wikipedia_tool()
search_govuk = build_govuk_search_tool()

agent_tool_names = ["_search_documents", "_search_wikipedia", "_search_govuk"]

tools = {
    "_search_documents": search_documents,
    "_search_govuk": search_govuk,
    "_search_wikipedia": search_wikipedia,
}
agent_tools = [tools[tool_name] for tool_name in agent_tool_names]

# agent_tools = [search_documents]
# agent_tools = [search_govuk]


# ToolNode will automatically take care of injecting state into tools
tool_node = ToolNode(agent_tools)
checkpointer = MemorySaver()

In [ ]:
agent_tools

In [ ]:
chat_backend = ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock")
llm = get_chat_llm(chat_backend, tools=agent_tools)

In [ ]:
config = {
    "es_client": _env.elasticsearch_client(),
    "index_name": _env.elastic_chunk_alias,
    "embedding_model": embedding_model,
    "embedding_field_name": _env.embedding_document_field_name,  # not used
    "chunk_resolution": ChunkResolution.normal,
}

In [ ]:
inputs = {
    "messages": [{"type": "user", "content": user_query}],
    "request": redbox_query,
    "config": config,
    "is_last_step": False,
    "remaining_steps": 1,
}

In [ ]:
inputs

In [ ]:
# optimized prompt to avoid hallucinationa
AGENTIC_RETRIEVAL_SYSTEM_PROMPT = (
    "You are an advanced problem-solving assistant. Your primary goal is to carefully "
    "analyse and work through complex questions or problems. You will receive a collection "
    "of documents (all at once, without any information about their order or iteration) and "
    "a list of tool calls that have already been made (also without order or iteration "
    "information). Based on this data, you are expected to think critically about how to "
    "proceed.\n"
    "\n"
    "Objective:\n"
    "1. Your task is to answer user queries."
    "2. Examine the available documents and tool calls:\n"
    "- Evaluate whether the current information is sufficient to answer the question.\n"
    "- Consider the success or failure of previous tool calls based on the data they returned.\n"
    "- Hypothesise whether new tool calls might bring more valuable information.\n"
    "\n"
    "3. Decide whether you can answer this question:\n"
    "- If additional tool calls are likely to yield useful information, make those calls.\n"
    "- Determine whether the available documents contain the answer to the question. If the available documents contain the answer to the question, provide an answer. You must provide the citations where you use the information to answer.\n"
    "- Determine whether the available documents contain the answer to the question. If the available documents do not contain the answer to the question, say I don't know. Do not make up an answer."
    "Your role is to think deeply before taking any action. Carefully weigh whether new "
    "information is necessary or helpful. Only take action (call tools or providing and answer) after "
    "thorough evaluation of the current documents and tool calls."
)

In [ ]:
graph = create_react_agent(
    llm, tools=agent_tools, state_schema=MyRedboxState, messages_modifier=AGENTIC_RETRIEVAL_SYSTEM_PROMPT, debug=True
)  # , checkpointer=checkpointer) #, checkpointer=checkpointer) #, state_modifier=prompt)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
print_stream(graph.stream(inputs, stream_mode="values"))

## LIMITATIONS

Stop criteria hasn't been implemented and might cause GraphRecursionError. 